In [1]:
# ==========================================
# 03 - EXPERIMENTO: RANDOM FOREST
# ==========================================

# 1. Configuración e Importaciones
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
!pip install opendatasets
import opendatasets as od

# 2. Carga de Datos
dataset_link="https://www.kaggle.com/competitions/udea-ai-4-eng-20252-pruebas-saber-pro-colombia/overview"
od.download(dataset_link)

data_path = "udea-ai-4-eng-20252-pruebas-saber-pro-colombia/"
train = pd.read_csv(data_path + "train.csv")
test = pd.read_csv(data_path + "test.csv")

# Guardamos los IDs del test para el archivo de envío final
test_ids = test['ID']

# 3. Separación de Features y Target
target_col = 'RENDIMIENTO_GLOBAL'

# Preparamos X e y
X = train.drop([target_col, 'ID'], axis=1)
y = train[target_col]
X_test = test.drop('ID', axis=1)

# 4. Preprocesado (Pipeline de la entrega anterior)
# Definimos listas de columnas
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# PERIODO_ACADEMICO a texto
if 'PERIODO_ACADEMICO' in numeric_features:
    numeric_features.remove('PERIODO_ACADEMICO')
    categorical_features.append('PERIODO_ACADEMICO')
    X['PERIODO_ACADEMICO'] = X['PERIODO_ACADEMICO'].astype(str)
    X_test['PERIODO_ACADEMICO'] = X_test['PERIODO_ACADEMICO'].astype(str)

# Pipelines de transformación
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 5. Codificación del Target (Etiquetas)
# Definimos el orden para que tenga sentido (ordinal)
categories_order = [['bajo', 'medio-bajo', 'medio-alto', 'alto']]
target_encoder = OrdinalEncoder(categories=categories_order)
y_encoded = target_encoder.fit_transform(y.to_frame()).ravel() # .ravel() para hacerlo array 1D

# 6. Definición del Modelo Completo (Pipeline + Modelo)
# Usamos RandomForestClassifier. n_estimators=100 es un estándar sólido.
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # Cambios realizados:
    # 1. n_estimators: Bajamos de 100 a 50 (mitad de tiempo)
    # 2. max_depth: Limitamos la profundidad a 15 (evita que los árboles crezcan infinito y coman RAM)
    # 3. max_features: 'sqrt' (es el default, pero nos aseguramos)
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        max_depth=25,
        min_samples_leaf=10,
        n_jobs=-1,
        random_state=42
    ))
])

# 7. Entrenamiento
print("Entrenando el modelo (esto puede tardar unos minutos)...")
model.fit(X, y_encoded)
print("¡Entrenamiento completado!")

# 8. Predicción sobre el Test Set
print("Generando predicciones sobre test.csv...")
y_pred_encoded = model.predict(X_test)

# 9. Decodificación y Generación del Submission
# Importante: Kaggle espera las etiquetas de texto (bajo, alto, etc.), no números (0, 1, 2, 3)
# Usamos inverse_transform para volver a texto
y_pred_labels = target_encoder.inverse_transform(y_pred_encoded.reshape(-1, 1)).ravel()

submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': y_pred_labels
})

# 10. Guardar archivo
submission.to_csv('submission_rf.csv', index=False)
print("Archivo 'submission_rf.csv' guardado exitosamente.")
print(submission.head())

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: simoncorrearios
Your Kaggle Key: ··········


100%|██████████| 29.9M/29.9M [00:00<00:00, 736MB/s]


Extracting archive ./udea-ai-4-eng-20252-pruebas-saber-pro-colombia/udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip to ./udea-ai-4-eng-20252-pruebas-saber-pro-colombia


Entrenando el modelo (esto puede tardar unos minutos)...
¡Entrenamiento completado!
Generando predicciones sobre test.csv...
Archivo 'submission_rf.csv' guardado exitosamente.
       ID RENDIMIENTO_GLOBAL
0  550236               alto
1   98545         medio-alto
2  499179               alto
3  782980               bajo
4  785185               bajo
